In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_excel('ChatGPT Evaluation v2.0.xlsx', sheet_name='7.WordContext')
df.head()

,index,target_word,PoS,index1-index2,example_1,example_2,label,split,prompt,chatgpt_raw,chatgpt_final
0,0,carry,V,2-1,You must carry your camping gear .,Sound carries well over water .,F,train,Are the given texts expressing the same sense ...,NaN,NaN
1,1,go,V,2-6,Messages must go through diplomatic channels .,Do you think the sofa will go through the door ?,F,train,Are the given texts expressing the same sense ...,NaN,NaN
2,2,break,V,0-2,Break an alibi .,The wholesaler broke the container loads into ...,F,train,Are the given texts expressing the same sense ...,NaN,NaN
3,3,cup,N,8-4,He wore a jock strap with a metal cup .,Bees filled the waxen cups with honey .,T,train,Are the given texts expressing the same sense ...,NaN,NaN
4,4,academy,N,1-2,The Academy of Music .,The French Academy .,F,train,Are the given texts expressing the same sense ...,NaN,NaN


In [ ]:
df = df[df['split'] == 'dev']

df['chatgpt_raw'] = df['chatgpt_raw'].astype(str)
df['chatgpt_final'] = df['chatgpt_final'].astype(str)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 638 entries, 5428 to 6065
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          638 non-null    int64 
 1   target_word    638 non-null    object
 2   PoS            638 non-null    object
 3   index1-index2  638 non-null    object
 4   example_1      638 non-null    object
 5   example_2      638 non-null    object
 6   label          638 non-null    object
 7   split          638 non-null    object
 8   prompt         638 non-null    object
 9   chatgpt_raw    638 non-null    object
 10  chatgpt_final  638 non-null    object
dtypes: int64(1), object(10)
memory usage: 59.8+ KB


<ipython-input-26-ed0e20565c09>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['chatgpt_raw'] = df['chatgpt_raw'].astype(str)
<ipython-input-26-ed0e20565c09>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['chatgpt_final'] = df['chatgpt_final'].astype(str)


In [ ]:
len(df.query('chatgpt_raw != chatgpt_final'))

6

In [ ]:
df['label'].value_counts()

F    319
T    319
Name: label, dtype: int64

In [ ]:
LABEL_MAP = {
    "T": 1,
    "F": 0,
}

gold_labels = [LABEL_MAP[val] for val in df.label.values]
chatgpt_preds = [LABEL_MAP[val] for val in df.chatgpt_final.values]

In [ ]:
from sklearn.metrics import classification_report

report_lines = classification_report(y_true=gold_labels, y_pred=chatgpt_preds, digits=4).split('\n')

for line in report_lines:
  print(line) 

              precision    recall  f1-score   support

           0     0.6079    0.8213    0.6987       319
           1     0.7246    0.4702    0.5703       319

    accuracy                         0.6458       638
   macro avg     0.6663    0.6458    0.6345       638
weighted avg     0.6663    0.6458    0.6345       638



In [ ]:
from collections import Counter

c = Counter(gold_labels)
list(c.values())

[319, 319]

In [ ]:
from scipy.stats import entropy
entropy(list(c.values()))

0.6931471805599453